In [33]:
import json
import re
import os
import sys, os
import pandas as pd
sys.path.append(os.path.abspath(os.path.join('../scr')))

In [34]:
emoji_pattern = re.compile("["
                                u"\U0001F600-\U0001F64F"  # Emoticons
                                u"\U0001F300-\U0001F5FF"  # Miscellaneous Symbols and Pictographs
                                u"\U0001F680-\U0001F6FF"  # Transport & Map Symbols
                                u"\U0001F700-\U0001F77F"  # Alchemical Symbols
                                u"\U0001F780-\U0001F7FF"  # Geometric Shapes Extended
                                u"\U0001F800-\U0001F8FF"  # Supplemental Arrows-C
                                u"\U0001F900-\U0001F9FF"  # Supplemental Symbols and Pictographs
                                u"\U0001FA00-\U0001FA6F"  # Chess Symbols
                                u"\U0001FA70-\U0001FAFF"  # Symbols and Pictographs Extended-A
                                u"\u2600-\u26FF"  # Miscellaneous Symbols
                                u"\u2700-\u27BF"  # Dingbats
                                u"\u2B50"  # Star
                                u"\U0001F1E6-\U0001F1FF"  # Country Flags
                                "]+", flags=re.UNICODE)
symbols = re.compile("["
                            "\""
                            "\“"
                            "\""
                            "\'"
                            "\-"
                            "\*"
                            "\•"
                            "\ℹ"
                            "\﻿"
                            "\_"
                            "]+")
url_pattern = r'http[s]?://(?:[a-zA-Z]|[0-9]|[$-_@.&+]|[!*\\(\\),]|(?:%[0-9a-fA-F][0-9a-fA-F]))+'
mention_pattern = r'@(\w+)'

In [44]:
def read_file(file_path: str) -> dict:
    # Open the file for reading
    with open(file_path, 'r') as file:
        # Load the JSON data from the file
        data = json.load(file)
        return data

In [45]:
def write_file(file_path: str, data: dict) -> None:
    # Open the file for writing
    with open(file_path, 'w') as file:
        # Dump the JSON data to the file
        json.dump(data, file, indent=2)

In [52]:

def parse_text(text: any) -> str:
    if isinstance(text, str):
        return text
    elif isinstance(text, list):
        contents = []
        for item in text:
            if isinstance(item, str):
                contents.append(item)
            elif isinstance(item, dict):
                contents.append(item['text'])
        return "".join(contents)
    else:
        return ""

In [53]:
def parse_messages(messages: list) -> dict:
    parsed_messages = {
        'id': [],
        'text': [],
        'date': []
    }
    for message in messages:
        if message['type'] != 'message' or len(message['text']) == 0:
            continue
        parsed_messages['id'].append(message['id'])
        text = parse_text(message['text'])
        parsed_messages['text'].append(text)
        parsed_messages['date'].append(message['date'])
    return parsed_messages

In [54]:
def extract_hashtags(text: str) -> list:
    return [word for word in text.split() if word.startswith('#')]

def extract_emojis(self, text):
    return ''.join(self.emoji_pattern.findall(text))

def remove_emojis(self, text):
    return self.emoji_pattern.sub('', text)

def extract_symbols(self, text):
    return ''.join(self.symbols.findall(text))

def remove_symbols(self, text):
    return self.symbols.sub(' ', text)

def extract_urls(self, text):
    return re.findall(self.url_pattern, text)

def extract_mentions(self, text):
    return re.findall(self.mention_pattern, text)

In [55]:
raw_dir = "../data/raw"
parsed_dir = "../data/parsed"
file_name = "4-3-3 FAST SPORT™"

In [56]:
data = read_file(f'{raw_dir}/{file_name}.json')

In [57]:
parsed_message = parse_messages(data['messages'])

In [58]:
# Convert the dictionary to a Pandas DataFrame
df = pd.DataFrame(parsed_message)
df.set_index('id', inplace=True)
df.head()

,text,date
id,,
30554,አርሰናል የታሚ አብርሀምን ፊርማ ስፐርስ እና ዌስትሀምን በልጦ ለማግኘት ...,2021-07-21T11:15:04
31040,ዶኒ ቫንዴቢክ ከቤቱ አይለቅም!\n\nየማንቸስተር ዩናይትዱ ኮከብ ዶኒ ቫን...,2021-07-22T07:29:56
31041,"1982 ""ኬን ቤትስ ቼልሲን በ 1 ሚ ፓ ገዛ\n\n2003 ኬን ቤትስ ቼል...",2021-07-22T07:29:56
31042,ሉይ ቫን ሃል የ ኔዘርላንድ ብሄራዊ ቡድን አሰልጣኝ ተደርጎ ተሹሙዋል የ ...,2021-07-22T07:29:56
31043,የፓውሎ ዲባላ ወኪል በአጥቂው የውል ማራዘሚያ ላይ ለመወያየት በወሩ መጨረ...,2021-07-22T07:29:56


In [59]:
df.to_csv(f'{parsed_dir}/{file_name}.csv')